# Key Idea  

## Client Side
- DPoP in headers when making request
- Signature is inside DPoP
- Signature = headers, claims, client private key
- headers contains alg, typ, jwk (public key in jwk format including e, kty, n in this exact order)
- claims contains jti, iat, exp (short live such as 15 to 30 seconds), htm, htu, client id (optional)

## Server Side
- Access Token = claims, server private key
- claims = jti, iat, exp (mid live such as 10 to 30 minutes), client id (optional), cnf.jkt (jwk from client's public key converting to thumbprint)
- Refresh Token = claims, server private key
- claims = jti, iat, exp (long live such as 24 hours or more), client id (optional)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time

In [3]:
from package.jwt_management.data_models.base_models import JWK
from package.jwt_management.data_models.server_models import ServerSignature, JKT, CNF
from package.jwt_management.data_models.client_models import ClientHeaders, ClientClaims, ClientSignature

In [4]:
from package.keypair_management import KeyPairManagement
km = KeyPairManagement(directory='./client_keypair')
skm = KeyPairManagement(directory='./server_keypair')
km.generate_keypairs()

Directory './client_keypair' already exists.


In [5]:
DPOP_LIVE = 15
ACCESS_TOKEN_LIVE = 60*10
REFRESH_TOKEN_LIVE = 60*24*1

In [6]:
from package.ezorm.db_management import delete_database, create_tables, delete_tables

In [7]:
delete_database()

In [8]:
from package.database_management.data_models import DPoPModel, AccessTokenModel, RefreshTokenModel
from package.ezorm.crud import Create, Read, Update, Delete

In [9]:
tables = [DPoPModel, AccessTokenModel, RefreshTokenModel]
create_tables(tables)

Model: dpopmodel created successfully
Model: accesstokenmodel created successfully
Model: refreshtokenmodel created successfully
All tables created successfully


# Client

In [10]:
jti = "test"
iat = int(time.time())
exp = iat + DPOP_LIVE
htm = "GET"
htu = "www.testapi.com/api/token"
client_id = "555"

In [11]:
ClientClaims(jti=jti, iat=iat, exp=exp, htm=htm, htu=htu, client_id=client_id)

ClientClaims(jti='test', iat=1733508804, exp=1733508819, htm='GET', htu='www.testapi.com/api/token', client_id='555')

In [12]:
ClientHeaders.from_key(key=km.load_public_key_from_pem())

ClientHeaders(typ=None, alg=None, jwk=JWK(e='AQAB', kty='RSA', n='q2yb3mO7hDAp6JZj2ijx536T8DWFA_ZNsYHl-enTTy6oOjTAWV2tVIht3g_X8xGZImBRSF1xtdXP0psoR9lKKyCZ4Z7jETN1fAtZNGYJNplEn-yCGaJQ4eCNRRX4Z24Ey0m0PlOjGppV9-3ws9VAVxc4kS0547Qq361VU-iHG_XrU8Jy97YKIX1xo0ie9FJtXKCEOsg0cpTXYQ_O5LrRt2cmm9TwDxXgbeCNnxNloH1bdM5BqkdYF4TYSL2Spphj9E78IkLJSkj_JfQyXlYe7iX2QibIc-Cw91qyw_zntFrCK5EkJQtDJnY6x8aGjMeexUzKJYUfJIvk3cYDm8W30w'))

In [13]:
c_sig = ClientSignature(
    headers=ClientHeaders(typ="dpop+jwt", alg="RS256", jwk=JWK.from_key(key=km.load_public_key_from_pem())),
    claims=ClientClaims(jti=jti, iat=iat, exp=exp, htm=htm, htu=htu, client_id=client_id)
)
# signature = c_sig.sign(key=km.load_private_key_from_pem())

In [16]:
c_sig.headers.jwk.to_thumbprint()

'SiBjGHhyEAh1M-x7FABlUoeLgnQB07rQlzEJhY7jJxA'

In [17]:
test = {"abc":1, "def":2}

In [19]:
assert "555" in test

AssertionError: 

In [15]:
headers = c_sig.headers.model_dump()

In [16]:
headers.pop('typ')

'dpop+jwt'

In [17]:
c_sig.headers = ClientHeaders(**headers)

In [18]:
c_sig

ClientSignature(headers=ClientHeaders(typ=None, alg='RS256', jwk=JWK(e='AQAB', kty='RSA', n='uB8_tLSevCqozv6yyO47hHRRyIuqa3NKw2IlDx4UY86bIjLKSh2PzcE85q9dvVTaBpLKM1DOXCuBHt3j9O6IJ4hPAoUqpt4WMcTYQmaETbn1w-qiYKpUmbp4ABdYeKteMIdUkkRQPg16KBtmMuM0G9K684XMgQBYbJnede1TkgP0oqE6eIDZzzanWP4b6GO4tAgZq9bsNLcrUAwrFt96KPX90W9vei8MxU8W457PSgni-qWzcdCqImh85iwZy3UvyOiJ_x13ZcSC-tZU00tj0vvHWsZKkxH2LXHTnsLVMQLhI-F9FaUusD-ojzmQhhHLSXKK3SmosoZod67FOhgpoQ')), claims=ClientClaims(jti='test', iat=1733506599, exp=1733506614, htm='GET', htu='www.testapi.com/api/token', client_id='555'))

In [19]:
c_sig.sign(key=km.load_private_key_from_pem())

'eyJhbGciOiJSUzI1NiIsImp3ayI6eyJlIjoiQVFBQiIsImt0eSI6IlJTQSIsIm4iOiJ1QjhfdExTZXZDcW96djZ5eU80N2hIUlJ5SXVxYTNOS3cySWxEeDRVWTg2YklqTEtTaDJQemNFODVxOWR2VlRhQnBMS00xRE9YQ3VCSHQzajlPNklKNGhQQW9VcXB0NFdNY1RZUW1hRVRibjF3LXFpWUtwVW1icDRBQmRZZUt0ZU1JZFVra1JRUGcxNktCdG1NdU0wRzlLNjg0WE1nUUJZYkpuZWRlMVRrZ1Awb3FFNmVJRFp6emFuV1A0YjZHTzR0QWdacTlic05MY3JVQXdyRnQ5NktQWDkwVzl2ZWk4TXhVOFc0NTdQU2duaS1xV3pjZENxSW1oODVpd1p5M1V2eU9pSl94MTNaY1NDLXRaVTAwdGowdnZIV3NaS2t4SDJMWEhUbnNMVk1RTGhJLUY5RmFVdXNELW9qem1RaGhITFNYS0szU21vc29ab2Q2N0ZPaGdwb1EifSwidHlwIjpudWxsfQ.eyJqdGkiOiJ0ZXN0IiwiaWF0IjoxNzMzNTA2NTk5LCJleHAiOjE3MzM1MDY2MTQsImh0bSI6IkdFVCIsImh0dSI6Ind3dy50ZXN0YXBpLmNvbS9hcGkvdG9rZW4iLCJjbGllbnRfaWQiOiI1NTUifQ.IsiWwsI1jqdJyN_C-OIazzQT0ySvdsgmsaeEbO56U1_2pfRS8W02W_Z1p7VF0DObW6WYu1hmItBSZ8Y7r7Ux7XhQ_9-m0Vo85e0tCo4zcn7VpifgVYykrxc0alqGPzfYqN3ZPuxv45wvAd8nsZVevAh7bjr0N20Yy9ZTQF8pBhIqQItIupEi-gpPk9PRE95BakWk4xfX4uj9DoHYVRtlHKw7nYjdq-H8R1KjmWXY5kPcNQPAImbFax8A3Ng_Ko_F8it0XEVxQE8jtjNcfWQBl3PvcAtYiFxpWnYpxUhNFurcnQkwb

In [ ]:
ClientSignature.sign(key=km.load_private_key_from_pem())

In [12]:
Create(DPoPModel(**c_sig.claims.model_dump()))

,Count
0,1


In [13]:
Read(DPoPModel())

,jti
0,test


In [14]:
c_sig.model_dump()

{'headers': {'typ': 'dpop+jwt',
  'alg': 'RS256',
  'jwk': {'e': 'AQAB',
   'kty': 'RSA',
   'n': 'tbRxtdgOHdYFx0V8CY0xjFxrxREm30wdiNtz5PJ_8iKSR0Tsuad1AnN98AkgFa5SCG4VyJyFU9ogpygqIhb497Y-w6TUaGWAHpttOALdVT9rbz30XBEcgE92KNQAhvVhD1z1iam1KIiqvN_hJ1NpvFqky4W0LxS8A9L4fjLSsRMXGGOUCTX-Qm2up4iaAsKFxZMIAaggiIeIkmNXMG36EOSpkog1CHTGZ_7AOIk2RP9w7e5HqHArbGvH2aWRW8BiNlygPwd0BI8QJSxpXsvjKgMw7DsNdkrjWOdPBZpQodpTSnjesKflLF3vnYQqqOSewK2Mc-c9kl7s-GorIfHnmw'}},
 'claims': {'jti': 'test',
  'iat': 1733491282,
  'exp': 1733491297,
  'htm': 'GET',
  'htu': 'www.testapi.com/api/token',
  'client_id': '555'}}

In [15]:
valid_signature = ClientSignature.verify_signature(signature=signature)
valid_signature

ClientSignature(headers=ClientHeaders(typ='dpop+jwt', alg='RS256', jwk=JWK(e='AQAB', kty='RSA', n='tbRxtdgOHdYFx0V8CY0xjFxrxREm30wdiNtz5PJ_8iKSR0Tsuad1AnN98AkgFa5SCG4VyJyFU9ogpygqIhb497Y-w6TUaGWAHpttOALdVT9rbz30XBEcgE92KNQAhvVhD1z1iam1KIiqvN_hJ1NpvFqky4W0LxS8A9L4fjLSsRMXGGOUCTX-Qm2up4iaAsKFxZMIAaggiIeIkmNXMG36EOSpkog1CHTGZ_7AOIk2RP9w7e5HqHArbGvH2aWRW8BiNlygPwd0BI8QJSxpXsvjKgMw7DsNdkrjWOdPBZpQodpTSnjesKflLF3vnYQqqOSewK2Mc-c9kl7s-GorIfHnmw')), claims=ClientClaims(jti='test', iat=1733491282, exp=1733491297, htm='GET', htu='www.testapi.com/api/token', client_id='555'))

In [16]:
try:
    ClientSignature.verify_signature(signature=signature).claims.validate_method_endpoint(method="GET", endpoint=htu)
except Exception as e:
    print(e)

In [17]:
try:
    ClientSignature.verify_signature(signature=signature).claims.validate_method_endpoint(method="POST", endpoint=htu)
except Exception as e:
    print(e)

400: http method mismatched


In [18]:
try:
    ClientSignature.verify_signature(signature=signature).claims.validate_method_endpoint(method="GET", endpoint="www.wrongendpoint/api/token")
except Exception as e:
    print(e)

400: http endpoiont mismatched


In [19]:
try:
    ClientSignature.verify_signature(signature=signature).claims.validate_method_endpoint(method="POST", endpoint="www.wrongendpoint/api/token")
except Exception as e:
    print(e)

400: http method and endpoint mismatched


In [20]:
jti = "test"
iat = int(time.time())
exp = iat + ACCESS_TOKEN_LIVE
client_id = "555"

In [21]:
jkt = JKT.from_key(key=valid_signature.headers.jwk.to_key())
jkt

JKT(jkt='s7uyr-kkOQMZBuJJeWxyG3j4rBKF2chcZhfolIUY1AU')

In [22]:
s_sig = ServerSignature(jti=jti, iat=iat, exp=exp, client_id=client_id, cnf=jkt)
access_token = s_sig.sign(key=skm.load_private_key_from_pem())

In [23]:
s_sig.model_dump()

{'jti': 'test',
 'iat': 1733491283,
 'exp': 1733491883,
 'client_id': '555',
 'cnf': {'jkt': 's7uyr-kkOQMZBuJJeWxyG3j4rBKF2chcZhfolIUY1AU'}}

In [24]:
ServerSignature.verify_signature(
    signature=access_token, 
    key=skm.load_public_key_from_pem()
).model_dump()

{'jti': 'test',
 'iat': 1733491283,
 'exp': 1733491883,
 'client_id': '555',
 'cnf': {'jkt': 's7uyr-kkOQMZBuJJeWxyG3j4rBKF2chcZhfolIUY1AU'}}

In [25]:
jti = "test"
iat = int(time.time())
exp = iat + REFRESH_TOKEN_LIVE
client_id = "555"

In [26]:
s_sig = ServerSignature(jti=jti, iat=iat, exp=exp, client_id=client_id)
refresh_token = s_sig.sign(key=skm.load_private_key_from_pem())

In [27]:
s_sig.model_dump()

{'jti': 'test',
 'iat': 1733491284,
 'exp': 1733492724,
 'client_id': '555',
 'cnf': None}

In [28]:
ServerSignature.verify_signature(
    signature=refresh_token, 
    key=skm.load_public_key_from_pem()
).model_dump()

{'jti': 'test',
 'iat': 1733491284,
 'exp': 1733492724,
 'client_id': '555',
 'cnf': None}

# DPoP Signature Verification Failed (Tampered)

In [29]:
jti = "test"
iat = int(time.time())
exp = iat + DPOP_LIVE
htm = "GET"
htu = "www.testapi.com/api/token"
client_id = "555"

In [30]:
c_sig = ClientSignature(
    headers=ClientHeaders.from_key(key=km.load_public_key_from_pem()),
    claims=ClientClaims(jti=jti, iat=iat, exp=exp, htm=htm, htu=htu, client_id=client_id)
)
c_signature = c_sig.sign(key=km.load_private_key_from_pem())

In [31]:
fc_sig = ClientSignature(
    headers=ClientHeaders.from_key(key=km.load_public_key_from_pem()),
    claims=ClientClaims(jti=jti, iat=iat, exp=exp, htm="POST", htu=htu, client_id=client_id)
)
fc_signature = fc_sig.sign(key=km.load_private_key_from_pem())

In [32]:
h, _, s = c_signature.split(".")
_, c, _ = fc_signature.split(".")

try:
    ClientSignature.verify_signature(signature=".".join([h,c,s]))
except Exception as e:
    print(e)

400: Unexpected error: Signature verification failed.


# ACCESS TOKEN Signature Verification Failed (Tampered)

In [33]:
jti = "test"
iat = int(time.time())
exp = iat + ACCESS_TOKEN_LIVE
client_id = "555"

In [34]:
jkt = JKT.from_key(key=valid_signature.headers.jwk.to_key())

In [35]:
s_sig = ServerSignature(jti=jti, iat=iat, exp=exp, client_id=client_id, cnf=jkt)
access_token = s_sig.sign(key=skm.load_private_key_from_pem())

In [36]:
Create(AccessTokenModel(access_token=access_token, active=True, remark="", **s_sig.model_dump()))

,Count
0,1


In [37]:
Read(AccessTokenModel(access_token=access_token, active=True, remark="", **s_sig.model_dump()))

,jti,access_token,client_id,exp,active,remark
0,test,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...,555,1733491886,True,


In [38]:
Update(
    existing=AccessTokenModel(access_token=access_token, active=True, remark="", **s_sig.model_dump()),
    new=AccessTokenModel(access_token=access_token, active=False, remark="tampered"),
)

,Count
0,1


In [39]:
Read(AccessTokenModel())

,jti,access_token,client_id,exp,active,remark
0,test,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...,555,1733491886,False,tampered


In [40]:
fs_sig = ServerSignature(jti="fake", iat=iat, exp=exp, client_id=client_id, cnf=jkt)
f_access_token = fs_sig.sign(key=skm.load_private_key_from_pem())

In [41]:
h, _, s = access_token.split(".")
_, c, _ = f_access_token.split(".")

In [42]:
try:
    ServerSignature.verify_signature(signature=".".join([h,c,s]), key=skm.load_public_key_from_pem())
except Exception as e:
    print(e)

400: Unexpected error: Signature verification failed.


In [43]:
s_sig.model_dump()

{'jti': 'test',
 'iat': 1733491286,
 'exp': 1733491886,
 'client_id': '555',
 'cnf': {'jkt': 's7uyr-kkOQMZBuJJeWxyG3j4rBKF2chcZhfolIUY1AU'}}

# REFRESH TOKEN Signature Verification Failed (Tampered)

In [44]:
jti = "test"
iat = int(time.time())
exp = iat + ACCESS_TOKEN_LIVE
client_id = "555"

In [45]:
jkt = JKT.from_key(key=valid_signature.headers.jwk.to_key())

In [46]:
s_sig = ServerSignature(jti=jti, iat=iat, exp=exp, client_id=client_id)
refresh_token = s_sig.sign(key=skm.load_private_key_from_pem())

In [47]:
Create(RefreshTokenModel(access_token=access_token, refresh_token=refresh_token, active=True, remark="", **s_sig.model_dump()))

,Count
0,1


In [48]:
Read(RefreshTokenModel(access_token=access_token, refresh_token=refresh_token, active=True, remark="", **s_sig.model_dump()))

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...,555,1733491890,True,,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...


In [49]:
Update(
    existing=RefreshTokenModel(access_token=access_token, refresh_token=refresh_token, active=True, **s_sig.model_dump()),
    new=RefreshTokenModel(access_token=access_token, refresh_token=refresh_token, active=False, remark="tampared"),
)

,Count
0,1


In [50]:
Read(RefreshTokenModel())

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...,555,1733491890,False,tampared,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...


In [51]:
fs_sig = ServerSignature(jti="fake", iat=iat, exp=exp, client_id=client_id)
f_refresh_token = fs_sig.sign(key=skm.load_private_key_from_pem())

In [52]:
h, _, s = refresh_token.split(".")
_, c, _ = f_refresh_token.split(".")

In [53]:
try:
    ServerSignature.verify_signature(signature=".".join([h,c,s]), key=skm.load_public_key_from_pem())
except Exception as e:
    print(e)

400: Unexpected error: Signature verification failed.


In [1]:
from fastapi import HTTPException

In [23]:
def test_idea():
    try:
        a
        # return a
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"{e}")

In [24]:
test_idea()

HTTPException: 400: name 'a' is not defined

In [25]:
try:
    test_idea()
except Exception as e:
    error = e
    

In [28]:
error.status_code, error.detail

(400, "name 'a' is not defined")

In [19]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'args',
 'with_traceback']

In [22]:
error.args

("HTTPException.__init__() got an unexpected keyword argument 'status'",)

In [30]:
from package.database_management.data_models import DPoPModel
from package.ezorm.crud import Read

In [31]:
Read(DPoPModel())

,jti
0,testreplay
1,1eedb0d0-c8c4-4453-9486-b92236253cb3
2,2b57d910-dbcb-45dd-a518-738807a6a96d
3,d7219d04-aeb8-4cf0-a87f-63c8af9a929f
4,7954c728-9f03-4471-b41f-fc3121f65b88
5,4d6e56c5-8066-4f80-8ca1-dccd9cc43edd
6,feeea30c-9976-47bb-a4c2-64eb4d1335e6
7,e154d135-6ce0-4143-865a-df6c961e8d55
8,8c3af36c-baa0-4098-9d19-9d5d88e3793d


In [33]:
import base64
import hashlib
import os

def generate_code_verifier():
    """Generate a random code verifier."""
    return base64.urlsafe_b64encode(os.urandom(32)).rstrip(b'=').decode('utf-8')

def generate_code_challenge(code_verifier):
    """Generate a code challenge based on the code verifier."""
    digest = hashlib.sha256(code_verifier.encode('utf-8')).digest()
    return base64.urlsafe_b64encode(digest).rstrip(b'=').decode('utf-8')

# Example
code_verifier = generate_code_verifier()
code_challenge = generate_code_challenge(code_verifier)
print(f"Code Verifier: {code_verifier}")
print(f"Code Challenge: {code_challenge}")


Code Verifier: L0k40jhcExb_CUEMs5z_-aML2j2t7R85FMdgVjYz6L8
Code Challenge: aUdMQ-V4vLWdMAEG2UHBknz8NSMx-ZvPY48F9-eGeM8


In [34]:
AUTHORIZATION_SERVER = "https://server.example.com"
CLIENT_ID = "s6BhdRkqt"
REDIRECT_URI = "https://client.example.com/cb"
SCOPE = "read_profile"

auth_url = (
    f"{AUTHORIZATION_SERVER}/authorize?response_type=code"
    f"&client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}"
    f"&scope={SCOPE}&code_challenge={code_challenge}&code_challenge_method=S256"
)

print(f"Visit this URL to log in: {auth_url}")


Visit this URL to log in: https://server.example.com/authorize?response_type=code&client_id=s6BhdRkqt&redirect_uri=https://client.example.com/cb&scope=read_profile&code_challenge=aUdMQ-V4vLWdMAEG2UHBknz8NSMx-ZvPY48F9-eGeM8&code_challenge_method=S256


In [1]:
from package.database_management.data_models import UserModel

In [2]:
import pandas as pd
user_data = pd.DataFrame({
    "client_id": ["555"],
    "username": ["555"],
    "password": ["555"],
})

In [8]:
user_data.iloc[0].to_dict()

{'client_id': '555', 'username': '555', 'password': '555'}

In [9]:
UserModel(**user_data.iloc[0].to_dict())

UserModel(client_id='555', username='555', password='555')

In [13]:
import base64
import os

# Generate 16 random bytes
random_bytes = os.urandom(16)

# Encode in URL-safe Base64
token = base64.urlsafe_b64encode(random_bytes).rstrip(b'=').decode('utf-8')
print(token)


hTpzPpSngbBsOoo0QORCdg


In [14]:
import hashlib
import os

# Generate 16 random bytes
random_bytes = os.urandom(16)

# Create SHA256 hash
hash_object = hashlib.sha256(random_bytes)
hashed_token = base64.urlsafe_b64encode(hash_object.digest()).rstrip(b'=').decode('utf-8')
print(hashed_token)


omMgQaCU-oZk0D502Si5P8UxVj3Pp2QU3lDPMQ_SUsA


In [15]:
import secrets
import base64
import os
import hashlib

# 1. Using secrets.token_urlsafe(16)
secrets_token = secrets.token_urlsafe(16)
print(f"secrets.token_urlsafe(16): {secrets_token}")

# 2. Using base64 with os.urandom(16)
random_bytes = os.urandom(16)
base64_token = base64.urlsafe_b64encode(random_bytes).rstrip(b'=').decode('utf-8')
print(f"base64 with os.urandom(16): {base64_token}")

# 3. Using hashlib (SHA256) with os.urandom(16)
hash_object = hashlib.sha256(random_bytes)
hashed_token = base64.urlsafe_b64encode(hash_object.digest()).rstrip(b'=').decode('utf-8')
print(f"hashlib SHA256 (with os.urandom(16)): {hashed_token}")

# ----------- Encoding and Decoding -----------

# Decode the base64 string from the second method
decoded_base64 = base64.urlsafe_b64decode(base64_token + '==')  # Adding padding
print(f"Decoded base64 token (original random bytes): {decoded_base64}")

# Encode back the decoded bytes to check if we can get the same result
re_encoded_base64 = base64.urlsafe_b64encode(decoded_base64).rstrip(b'=').decode('utf-8')
print(f"Re-encoded base64 token (to match): {re_encoded_base64}")

# Test decoding the secrets token (no need for padding)
decoded_secrets_token = base64.urlsafe_b64decode(secrets_token + '==')
print(f"Decoded secrets token: {decoded_secrets_token}")

# Test decoding the hashed token (hashed tokens can't be decoded back, this is just for demonstration)
try:
    decoded_hashed_token = base64.urlsafe_b64decode(hashed_token + '==')
    print(f"Decoded hashed token: {decoded_hashed_token}")
except Exception as e:
    print(f"Error decoding hashed token: {e}")



secrets.token_urlsafe(16): WLcKHVU9o1in9CHlje6bOw
base64 with os.urandom(16): bU6iYZDXUgCmSVFX8P4qKA
hashlib SHA256 (with os.urandom(16)): 7v4TSmqCLJj_uY1ePiZR4_IjZOow2PMDq_qKsav-Umc
Decoded base64 token (original random bytes): b'mN\xa2a\x90\xd7R\x00\xa6IQW\xf0\xfe*('
Re-encoded base64 token (to match): bU6iYZDXUgCmSVFX8P4qKA
Decoded secrets token: b'X\xb7\n\x1dU=\xa3X\xa7\xf4!\xe5\x8d\xee\x9b;'
Decoded hashed token: b'\xee\xfe\x13Jj\x82,\x98\xff\xb9\x8d^>&Q\xe3\xf2#d\xea0\xd8\xf3\x03\xab\xfa\x8a\xb1\xab\xfeRg'


In [16]:
secrets_token

'WLcKHVU9o1in9CHlje6bOw'

In [17]:
from pydantic import BaseModel
from package.ezorm.variables import EzORM

In [19]:
class Test(BaseModel,EzORM):
    t:str

TypeError: Cannot create a consistent method resolution
order (MRO) for bases BaseModel, EzORM

In [22]:
from pydantic import BaseModel, create_model, Field

# Dynamically creating a Pydantic model using `create_model`
dym = create_model(
    'DynamicPerson',
    name=(str, Field(..., description="Name of the person")),
    age=(int, Field(..., description="Age of the person"))
)

# Create an instance of the dynamically created model
# person = DynamicModel(name="Alice", age=30)
# print(person)


In [23]:
dym

__main__.DynamicPerson

In [21]:
DynamicModel

__main__.DynamicPerson

In [34]:
from package.database_management.data_models import CodeModel, UserModel
from package.ezorm.crud import Read

In [35]:
Read(UserModel())

,client_id,username,password
0,24e5e6fe-c45e-4717-925c-a5aecdc3d584,string,string


In [36]:
Read(CodeModel())

,client_id,code_challenge,code
0,24e5e6fe-c45e-4717-925c-a5aecdc3d584,string,0_QC3DNti2iikKYaQIq0Xw
